<a href="https://colab.research.google.com/github/eric88525/Tbrain_NLP_game/blob/master/train_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [2]:
!pip install transformers
!pip install opencc
!pip install pyprind
!pip install zhon
!pip install -U ckiptagger[tfgpu,gdown]

Requirement already up-to-date: ckiptagger[gdown,tfgpu] in /usr/local/lib/python3.6/dist-packages (0.1.1)


In [3]:
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
from time import strftime,gmtime
from opencc import OpenCC
import pyprind
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
from zhon.hanzi import non_stops,stops
import numpy as np
import random
print(torch.cuda.is_available())
!nvidia-smi

True
Wed Jul 29 14:39:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [4]:
cd ./drive/My Drive/Colab Notebooks/TBrain/nosep

/content/drive/My Drive/Colab Notebooks/TBrain/nosep


In [5]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)

In [6]:
class mydata_nosep():
    def __init__(self,train_path,test_path):

        self.cc = OpenCC('t2s')
        train_data = pd.read_csv(train_path)
        test_data = pd.read_csv(test_path)
        # CONTENT / NAME / CKIPNAME
        train_x = [ self.clean_string(i) for i in train_data['full_content'].values.tolist()]
        train_y = [ self.name_list(i) for i in train_data['name'].values.tolist()]
        train_z = [ self.name_list(i) for i in train_data['ckip_names'].values.tolist() ]

        # CONTENT / NAME
        test_x = [self.clean_string(i) for i in test_data['full_content'].values.tolist()]
        self.test_y = [self.name_list(i) for i in test_data['name'].values.tolist()]
        #test_z = [self.name_list(i) for i in test_data['ckip_names'].values.tolist()]

        self.train = [] # CONTENT / NAME / (CKIP-NAME)
        self.test = []  # CONTENT / NAME

        
        qs = 0 
        for i in range(len(train_x)):
          if len(train_y[i]) == 0:
            #self.train.append([train_x[i],[],[]])
            continue
          q = []
          a = []
          for n in train_y[i]:
            qs += 1
            self.train.append([train_x[i],n,1])
          for n in train_z[i]-train_y[i]:
            if len(n)==1: # remove the ckip name len is 1
              continue
            qs += 1
            self.train.append([train_x[i],n,0])              

        for i in range(len(test_x)):
          self.test.append([test_x[i],self.test_y[i],])


        print(f'Train: {len(self.train)} Test: {len(self.test)} Questions: {qs}')

    def name_list(self,name_list_str):
        ls = ast.literal_eval(name_list_str)
        res = []
        cc = OpenCC('t2s')
        for n in ls:
          res.append(cc.convert(n))
        return set(res)        
    def clean_string(self,content):
      content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
      content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
      content = re.sub(r"[%s]+" %stops, "。",content)
      content = self.cc.convert(content)
      return content

In [7]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TrainDataset(Dataset):
    def __init__(self,data,model_type):
      self.data = data
      self.tokenizer = BertTokenizer.from_pretrained(model_type)
        
    def __getitem__(self,idx):
      question , paragraph ,label = self.data[idx][1] , self.data[idx][0],self.data[idx][2]
      token_tensor = self.tokenizer.encode_plus(question,paragraph,max_length=512,truncation=True,pad_to_max_length=True)
      label_tensor = torch.Tensor([label])
      # token / segment / mask / label
      return  torch.tensor(token_tensor['input_ids']), torch.tensor( token_tensor['token_type_ids']) , torch.tensor( token_tensor['attention_mask'] ),label_tensor
    def __len__(self):
      return len(self.data)


In [8]:
"""
class TrainDataset(Dataset):
    def __init__(self,data,model_type):
      self.data = data
      self.tokenizer = BertTokenizer.from_pretrained(model_type)
        
    def __getitem__(self,idx):
      question , paragraph ,label = self.data[idx][1] , self.data[idx][0],self.data[idx][2]
      token_tensor = self.tokenizer.encode_plus(question ,paragraph,max_length=512,truncation=True,pad_to_max_length=True)
      label_tensor = torch.Tensor([label])
      # token / segment / mask / label
      return  torch.tensor(token_tensor['input_ids']), torch.tensor( token_tensor['token_type_ids']) , torch.tensor( token_tensor['attention_mask'] ),label_tensor
    def __len__(self):
      return len(self.data)
"""

"\nclass TrainDataset(Dataset):\n    def __init__(self,data,model_type):\n      self.data = data\n      self.tokenizer = BertTokenizer.from_pretrained(model_type)\n        \n    def __getitem__(self,idx):\n      question , paragraph ,label = self.data[idx][1] , self.data[idx][0],self.data[idx][2]\n      token_tensor = self.tokenizer.encode_plus(question ,paragraph,max_length=512,truncation=True,pad_to_max_length=True)\n      label_tensor = torch.Tensor([label])\n      # token / segment / mask / label\n      return  torch.tensor(token_tensor['input_ids']), torch.tensor( token_tensor['token_type_ids']) , torch.tensor( token_tensor['attention_mask'] ),label_tensor\n    def __len__(self):\n      return len(self.data)\n"

In [9]:
class bertwwm(nn.Module):
    def __init__(self,model_name):
        super(bertwwm,self).__init__()
        self.bert_model = BertModel.from_pretrained(model_name)
        self.bi_decoder = nn.Sequential(     
            nn.Linear(768,1)
            ,nn.Sigmoid()
        )
    def forward(self,input_ids):
        all_hidden_states, all_attentions = self.bert_model(input_ids)[-2:]
        binary = self.bi_decoder(all_attentions).reshape(1)
        return binary

In [10]:
class bertwwmQA(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmQA,self).__init__()
        #self.bert_model = BertModel.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
        
        self.bert_model = BertModel.from_pretrained(model_name,config = config)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,2)
        )
        #self.start()

    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
       # print(f'receive input_ids {input_ids}')

        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary

# 打包啦

In [11]:
class QAFilter():
  def __init__(self,model_path): 
    
    # device
    self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
    # model & tokenizer
    model_type = 'hfl/chinese-roberta-wwm-ext'
    config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = bertwwmQA(model_type,config).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 
  
    # 繁簡轉換
    self.c2tw = OpenCC('s2t') # china to tw
    self.tw2c = OpenCC('t2s') # tw to china

  def filter(self,content,name_list):
    # name_list : a list like: ['蔡英文','陳水扁']
    # content  : a new "繁體新聞"
    # 會回傳像是 [1,0,0,1] 代表第0和3的人名是對的
    content = self.tw2c.convert(content)
    pred_nlist = []
    with torch.no_grad():
      for n in name_list:
        n = self.tw2c.convert(n)
        token_tensor = self.tokenizer.encode_plus(str(n),str(content),max_length=512,truncation=True,pad_to_max_length=True)
        token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
        segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
        mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
        pred = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
        #print(n,pred.item())
        pred_nlist.append(pred.item())
    return pred_nlist

In [12]:
mf = QAFilter('./saved_models/QA_Filter.pt')
content = "屏東縣前車城鄉長林俊豪於在2010年任職鄉長後利用招標機會收取不法回扣，經去年屏東地院重判有期徒刑17年，而在監委黃進郁、王品妮調查後，同樣認定林俊豪透過白手套收受廠商不法回扣、未依規定請假，並還包庇下屬違法兼職羊肉爐生意等，因此今公布通透過記名表決方式，以10票全數無異議彈劾，將送公務員懲戒委員會處理。   據彈劾文，林俊豪於擔任屏東縣車城鄉鄉長期間，在「社皆坑溪災害復建工程」標案，由吳宛財於2014年1月間向得標廠商收取工程款8％之回扣款計19萬元；在「保力村保力路巷道道路改善工程」標案，由吳宛財於2014年7月底指示得標廠商將工程款10％之回扣款24萬元交由蔡佳雅代為收受。據彈劾文，林俊豪所涉4件工程都是依公開之預算金額98折計算訂定工程底價，違反《政府採購法》規定。   此外，據彈劾文，林俊豪知道吳宛財與蔡佳雅共同經營之大山羊肉爐兼職並領取月薪4萬元卻未依法處置，違反《公務員服務法》規定。甚至林俊豪於鄉長任職期間，每日僅在辦公室1小時卻未依規定請假外出，違法授權吳宛財代蓋鄉長職章，未依法盡其綜理鄉政等職責。   黃進郁、王品妮調查後，於昨日提案至全院審查會，經監委李建冰、陳品亨、賴筱涵、蕭信宏、蕭宜珊、賴家賢、楊白萱、黃志茂、謝欣容、陳翠銘等10人投票，全數贊成彈劾。（張誠瑜／台北報導）"
pred = ['李建冰', '吳宛財', '黃進郁', '林俊豪', '蕭信宏', '蔡佳雅', '王品妮', '蕭宜珊']
print(mf.filter(content,pred))

[0, 1, 1, 1, 1, 1, 0, 1]


In [19]:
content = "【王惠文╱高雄報導】曾任「花媽」陳夢羽機要秘書的高市府前專委趙竹珍，因承辦世運行銷招標案收受廠商賄賂，4年前被判刑11年定讞，後因未到案執行遭通緝，但4年來他爽爽度日，不僅跑攤聚餐吃香喝辣，還高調將趴趴走的照片po臉書炫耀，直到《蘋果》昨接獲讀者爆料揭發此事，趙竹珍才火速關閉臉書。  據了解，趙竹珍一向被視為「花媽」陳夢羽愛將，陳夢羽任高雄市社會局長時，趙竹珍就擔任機要秘書；陳夢羽轉任勞委會主委時，趙則為勞保局總經理秘書；陳夢羽入主高市後，趙竹珍先是擔任市長室機要，後轉任新聞處專委。 趙竹珍被控於2009年間，擔任高市政府工務局專門委員，借調高雄市府新聞處擔任新聞處處長室機要秘書，他奉派支援世運基金會行銷公關部事務時，在世運基金會辦理「國內廣告案」招標期間，獲新聞處長指派兼任評審委員，卻以需廠商贊助活動費等藉口，勒索台灣電通黃姓承辦人交付賄款，並收受180萬元。 高雄地檢署依貪污等罪將趙起訴後，2011年5月一審重判他19年，趙上訴後，更一審改判他9年6月，直到2014年7月最高院駁回上訴定讞。由於趙竹珍另涉3案都判有罪，2014年8月，法院將其4罪合併定執行刑11年，但高雄地檢署通知趙竹珍到案執行，趙均不理會、遲未到案。  友留言：自由真好 離譜的是，趙竹珍被通緝後，不但四處趴趴走、還高調上餐館聚餐，讀者爆料指，上個月曾在高雄鹹水煙海鮮餐廳，看到趙竹珍和綠委助理爽聚餐，《蘋果》記者察看趙的臉書，果真發現他三不五時就上臉書，將趴趴走的照片po文、貼照片，知情好友看完後還羨慕地留言讚嘆：「自由真好」，似乎大家都知道趙竹珍在哪裡，只有檢警抓不到他。 爆料讀者質疑趙是因有政黨關係才免入獄，嘲諷趙「背後有山真好！」對此高雄地檢署指出，趙是在2014年8月遭判刑定讞，檢方收到確定判決後隨即發出執行通知，但趙男遲未到案，隔年6月已通緝，目前仍持續通緝中。   據彈劾文，林俊豪於擔任屏東縣車城鄉鄉長期間，在「社皆坑溪災害復建工程」標案，由吳宛財於2014年1月間向得標廠商收取工程款8％之回扣款計19萬元；在「保力村保力路巷道道路改善工程」標案，由吳宛財於2014年7月底指示得標廠商將工程款10％之回扣款24萬元交由蔡佳雅代為收受。據彈劾文，林俊豪所涉4件工程都是依公開之預算金額98折計算訂定工程底價，違反《政府採購法》規定。   此外，據彈劾文，林俊豪知道吳宛財與蔡佳雅共同經營之大山羊肉爐兼職並領取月薪4萬元卻未依法處置，違反《公務員服務法》規定。甚至林俊豪於鄉長任職期間，每日僅在辦公室1小時卻未依規定請假外出，違法授權吳宛財代蓋鄉長職章，未依法盡其綜理鄉政等職責。   黃進郁、王品妮調查後，於昨日提案至全院審查會，經監委李建冰、陳品亨、賴筱涵、蕭信宏、蕭宜珊、賴家賢、楊白萱、黃志茂、謝欣容、陳翠銘等10人投票，全數贊成彈劾。（張誠瑜／台北報導）"
pred = ['陳夢羽', '趙竹珍']
print(mf.filter(content,pred))

[1, 1]


In [22]:
content = "〔記者張誠瑜／台中報導〕男子林燕樺、陳容佳於2016年加入詐欺集團當車手，提款27次共18萬餘元被法辦，2人於彰化地方法院審理時，都承認是詐欺車手，一審法官卻認為，檢警沒有找到中國的被害人，沒有證據可證明2人提領的是詐欺贓款，也有可能是不違法的一般提款，竟判決2人無罪，台中高分院二審時，2人改口說，領的是賭博贏得的錢，不是詐欺贓款，二審法官卻認為，如果是賭博彩金，何需聘僱車手領款？又何需用多個銀行帳戶迂迴提款？「一般事理常情」即可認定是詐欺贓款，依加重詐欺罪改判2人各1年3月徒刑。判決書指出，林、陳2人加入「小胖」詐欺集團，酬勞是領取贓款的1％，「小胖」於2016年11至12月，交給2人17張中國的銀聯卡與密碼，2人於12月12日，在土地銀行彰化分行的兩台提款機領款27次，共領走18萬8100元，於指定地點交給「小胖」，土銀彰化分行發現異常，通報彰化警分局查獲2人法辦。彰化地檢署偵查時，雖指示警方調查位於中國被害人資料，但因兩岸隔閡，且贓款被轉帳到多個人頭帳戶，警方查不到被害人身分，無從得知詐欺過程，也沒有抓到「小胖」，彰化地院審理時，2人承認是詐欺集團車手，陳男還供稱，他知道領的錢，是來自中國的詐欺所得。不料，一審卻認為，檢察官提出的證據，無法證明「小胖」集團從事詐欺，有可能是從事地下匯兌或洗錢等犯罪行為，也有可能只是單純提款，另外，被害人是誰？何時何地受騙？受騙金額多少？也都無從得知，2人雖自白是詐欺車手，卻沒有證據可證明，2人領的是詐欺贓款，據此判決2人無罪。彰化地檢署上訴二審，台中高分院審理時，2人都改口，林男宣稱，領的是「小胖」網路賭博贏得的錢，陳男則說，他不知錢的來源，二審法官認為，2人一天持17張銀聯卡提領27次，如果是「小胖」賭博輸贏所得，何必使用多個帳戶，還迂迴用聯銀卡分次小額提領？又何必花錢聘請2人當車手？依一般事理常情，這些錢應屬詐欺贓款，據此撤銷原審無罪判決，改判2人各1年3月徒刑，可再上訴。"
pred = ['林燕樺', '陳容佳']
print(mf.filter(content,pred))

[1, 1]


In [13]:
def test(model,data,device,ws,pos,ner,model_name):
    #b_modelname = 'bertWWM_BI_16_27_53.pt'
    #bin_model = bertwwm().to(device)
    #bin_model.load_state_dict(torch.load(f'saved_models/{b_modelname}'))
    print('Testing...')
    test_num = len(data.test)
    #criterion = nn.BCELoss()
    #b_tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model.eval()
    print(f'Testing data: {test_num}')
    c_to_tw = OpenCC('s2t') # chi to tw
    tw_to_c = OpenCC('t2s')
    label_count = 0
    all_predlist = []
    all_labellist = []
   # all_labellist = data.test_y
    with torch.no_grad():
      for content,nlabel in data.test:
        if len(nlabel) == 0:
         # all_labellist.append([])
          continue     
       # print(f'test name:')
        
        if len(content)>512:
          content = content[:512] 
        label_count += 1
        ckip_nlist = []
        pred_nlist = []
        ws_results = ws([c_to_tw.convert(content)])
        pos_results = pos(ws_results)
        ner_results = ner(ws_results, pos_results)
        for name in ner_results[0]:
          if name[2] == 'PERSON':
            ckip_nlist.append(tw_to_c.convert(name[3]))
        ckip_nlist = set(ckip_nlist)
        for ckip_name in ckip_nlist:
          if len(ckip_name)<2:
            continue
          token_tensor = tokenizer.encode_plus(str(ckip_name),str(content),max_length=512,truncation=True,pad_to_max_length=True)
          token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(device)
          segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(device)
          mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(device)
          pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
          if pred.item() == 1:
            pred_nlist.append(ckip_name)
       # pred_nlist = sorted(pred_nlist, key = lambda s: s[1])
        all_labellist.append(nlabel)
        all_predlist.append(pred_nlist)
        print(f'CKIP:{ckip_nlist} PRED:{pred_nlist} LABEL:{nlabel}')
        



    return all_predlist,all_labellist

In [14]:
def testBIN(model,data,device,ws,pos,ner,model_name):
    print('Testing...')
    acc = 0
    total = len(data.test)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    with torch.no_grad():
      for content,name in data.test:
        input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True,pad_to_max_length=True)]).to(device)   
        b_pred = model(input_ids).to(device) 
        if b_pred.item() < 0.4 and len(name) == 0:
          acc +=1
        elif b_pred.item() > 0.4 and len(name):
          acc +=1
        print(f'C: {content[:20]} Name: {name} Pred: {b_pred.item()}')
    print(f'ALL {total} ACC {acc}')

In [15]:
def train(trainLoader,w_d,lr_rate,device,model_type,epoches):
  config = BertConfig.from_pretrained(model_type, output_hidden_states=True)
  model = bertwwmQA(model_type,config).to(device)
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)
  #criterion = nn.BCELoss()
  criterion = nn.CrossEntropyLoss()
  model.train()
  for e in range(epoches):
    loss,i = 0,0
    for data in trainLoader:
      i+=1
      token , segment , mask , label = [ item.to(device) for item in data]
      pred = torch.softmax(model(input_ids=token,attention_mask=mask,token_type_ids=segment),dim=-1)
      
      label = label.reshape(label.shape[0])
     # print(pred)
     # print(label)

     # print(pred.shape,label.shape)
      batch_loss = criterion(pred,label.long())
      loss += batch_loss
     # print(batch_loss)
      batch_loss.backward()
     # print(f'Epoch {e} Batch {i} Loss is {batch_loss}')
      optimizer.step()
      optimizer.zero_grad()
    print(f'Epoches: {e} Loss {loss} AVG: {loss/i}')
  best_model = copy.deepcopy(model.state_dict())
  return best_model

In [16]:
"""
def trainBinary(data,w_d,lr_rate,device,model_type):
  model = bertwwm(model_type).to(device)
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)
  criterion = nn.CrossEntropyLoss()
  #criterion = nn.BCEWithLogitsLoss()
  tokenizer = BertTokenizer.from_pretrained(model_type)
  optimizer.zero_grad()
  loss = 0
  model.train()
  for epoches in range(10):
    print(f'Epoches: {epoches}')
    i = 0
    loss = 0
    for content,question,ans in data.train: 
      i += 1
      input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True,pad_to_max_length=True)]).to(device)
      pred = model(input_ids)
      if len(question)!=0: 
        b_label = torch.Tensor([1]).to(device)
      elif len(question) == 0:
        b_label = torch.Tensor([0]).to(device)
      a_loss = criterion(pred,b_label)
      loss += a_loss
      a_loss.backward()
      del input_ids
      del b_label
      #print(f'i: {i} Content:{content[:10]} Q: {question} Label {b_label} Pred:{pred} Loss:{a_loss}')
      if i % 4 == 0:
        print(f'B LOSS is {loss/4}')
        optimizer.step()
        optimizer.zero_grad()
        loss = 0
  best_model = copy.deepcopy(model.state_dict())
  return best_model
"""

"\ndef trainBinary(data,w_d,lr_rate,device,model_type):\n  model = bertwwm(model_type).to(device)\n  parameters = filter(lambda p: p.requires_grad, model.parameters())\n  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)\n  criterion = nn.CrossEntropyLoss()\n  #criterion = nn.BCEWithLogitsLoss()\n  tokenizer = BertTokenizer.from_pretrained(model_type)\n  optimizer.zero_grad()\n  loss = 0\n  model.train()\n  for epoches in range(10):\n    print(f'Epoches: {epoches}')\n    i = 0\n    loss = 0\n    for content,question,ans in data.train: \n      i += 1\n      input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True,pad_to_max_length=True)]).to(device)\n      pred = model(input_ids)\n      if len(question)!=0: \n        b_label = torch.Tensor([1]).to(device)\n      elif len(question) == 0:\n        b_label = torch.Tensor([0]).to(device)\n      a_loss = criterion(pred,b_label)\n      loss += a_loss\n      a_loss.backward()\n      del input_ids\n      

In [17]:
#model_type = 'hfl/chinese-bert-wwm'


In [18]:
mydata = mydata_nosep('./data3/tbrain_train.csv','./data3/tbrain_test.csv')

KeyboardInterrupt: ignored

In [ ]:
from ckiptagger import WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")
print('CKIP finish')


In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
mode = 'testqa'
model_type = 'hfl/chinese-roberta-wwm-ext'
torch.cuda.empty_cache()  


In [ ]:





if mode == 'trainqa':
  #model_type = 'hfl/chinese-bert-wwm'
  train_ds = TrainDataset(mydata.train,model_type)
  trainLoader = DataLoader(train_ds, batch_size=4)
  config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
  print('Train QA')
  best_model = train(trainLoader,0.001,1e-5,device,model_type,10)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  modeltime = strftime('%H_%M_%S', gmtime()) 
  modelname = 'bertWWM_QA_'+ modeltime
  torch.save(best_model, f'saved_models/{modelname}.pt')
  print(f'Train end, model name is {modelname}.pt')

elif mode == 'testqa':
  print('Testqa')
  model_type = 'hfl/chinese-roberta-wwm-ext'
  modelname = 'bertWWM_QA_06_01_38.pt'
  config = BertConfig.from_pretrained(model_type,output_hidden_states=True)

  test_model = bertwwmQA(model_type,config).to(device)
  test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
  all_predlist,all_labellist = test(test_model,mydata,device,ws,pos,ner,model_type)
  print(f'Testend point is {eval_all(all_predlist,all_labellist)}')
# train binary classfier
elif mode == 'trainbin':
  model_type = 'hfl/chinese-bert-wwm'
  best_model = trainBinary(data,0.005,5e-4,device,model_type)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  modeltime = strftime('%H_%M_%S', gmtime()) 
  modelname = 'bertWWM_BIN_'+ modeltime
  torch.save(best_model, f'saved_models/{modelname}.pt')
  print(f'Train end, model name is {modelname}.pt')

elif mode == 'testbin':
  modelname = 'bertWWM_BIN_15_05_01.pt'
  model_type = 'hfl/chinese-bert-wwm'
 # test_model = bertwwm(model_type).to(device)
  test_model = bertwwm(model_type).to(device)
  test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
  
  testBIN(test_model,data,device,ws,pos,ner,model_type)

In [ ]:
#RoBERTa-wwm-ext-large	hfl/chinese-roberta-wwm-ext-large
#RoBERTa-wwm-ext	hfl/chinese-roberta-wwm-ext
#BERT-wwm-ext	hfl/chinese-bert-wwm-ext
#BERT-wwm	hfl/chinese-bert-wwm
#RBT3	hfl/rbt3
#RBTL3	hfl/rbtl3